In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import numpy as np
import seaborn as sns
%matplotlib inline
import tensorflow as tf
import keras
from sklearn.metrics import f1_score
import cv2
from tqdm import tqdm
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Convolution2D, MaxPooling2D, Input, GlobalAveragePooling2D
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical
from keras.applications.resnet50 import ResNet50
from sklearn.utils import shuffle
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.inception_v3 import InceptionV3
from keras.optimizers import Adam
from tensorflow.python.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping,CSVLogger, TensorBoard
from numpy.random import seed
seed(42)
from numpy.random import seed
seed(42)
from keras.applications.vgg16 import VGG16



Using TensorFlow backend.


In [2]:
datapath = "dataset/"
img_width, img_height = 224,224
classes = ["manipuri", "bharatanatyam", "odissi", "kathakali", "kathak", "sattriya", "kuchipudi", "mohiniyattam"]
class_names_label = {classes:i for i, classes in enumerate(set(classes))}
num_classes = len(classes)
model_try='vgg'

In [3]:
def load_images(path):
    img = cv2.imread(path)
    resized = cv2.resize(img, (img_width, img_height),interpolation=cv2.INTER_CUBIC)
    return resized

# def norm_images(images):
#     images = np.array(images,dtype = 'float32') / 255.  
#     mean_img = np.mean(images,axis = 0)
#     std_img = np.std(images,axis = 0)
#     norm_img = (images - mean_img)/std_img
#     return norm_img
#     return images

def load_dataset(folder_name):
    images = []
    labels = []
    label_file = pd.read_csv(os.path.join(datapath,folder_name+".csv"))
    for file in tqdm(os.listdir(os.path.join(datapath,folder_name))):
        if(folder_name == "train"):
            label = class_names_label[label_file[label_file["Image"] == file]["target"].values[0]]
            labels.append(label)
        else:
            labels.append(file)
        image = load_images(os.path.join(datapath,folder_name,file))
#         image = norm_images(image)
        images.append(image)
    images = np.array(images,dtype = 'float32')
#     images = norm_images(images)
    return (images,labels)

In [4]:
def show_batch(image_batch, label_batch):
    y_maps = dict()
    y_maps = {v:k for k,v in class_names_label.items()}
#     print(y_maps)
    plt.figure(figsize=(12,12))
    for n in range(25):
        ax = plt.subplot(5,5,n+1)
        plt.imshow(image_batch[n])
        pred = np.argmax(label_batch[n])
#         print(pred)
        plt.title(y_maps[pred])
        plt.axis('off')

In [5]:
def generate_sol(name, model):
    predictions = model.predict(x_test)
    predictions = np.argmax(predictions,axis = 1)
    y_maps = dict()
    y_maps = {v:k for k,v in class_names_label.items()}
    print(y_maps)
    print(predictions)
    pred_labels = [y_maps[k] for k in predictions]
    print(pred_labels)
    sub = pd.DataFrame({'Image':img_name,'target':pred_labels})
#     model.save('weights/'+name+'weights.h5')
    sub.to_csv('solutions_experiment/'+name+'sol.csv', index = False)

In [6]:
(x_train, y_train)=load_dataset("train")
x_train,y_train = shuffle(x_train,y_train,random_state = 42)
(x_test, img_name) = load_dataset("test")
y_train = np.array(y_train, dtype = 'int16')
y_train = to_categorical(y_train)

print(y_train.shape)

100%|██████████| 156/156 [00:00<00:00, 228.97it/s]

(364, 8)


In [ ]:
show_batch(x_train,y_train)

In [7]:
datagen = ImageDataGenerator(rotation_range = 40, width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=[0.2,1.0],
        horizontal_flip=True,
        fill_mode='nearest',
        validation_split=0.3,
        brightness_range = [0.2,1.3])
#val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = datagen.flow(x_train, y_train,subset="training",batch_size=32)
val_generator = datagen.flow(x_train, y_train,subset = "validation", batch_size=32)
#val_generator = val_datagen.flow(validation_imgs, validation_labels_enc, batch_size=30)

In [ ]:
x_batch, y_batch = next(train_generator)
for i in range (0,32):
    y_maps = dict()
    y_maps = {v:k for k,v in class_names_label.items()}
#   
    image = x_batch[i]
    plt.title(y_maps[np.argmax(y_batch[i])])
#     print(image.shape)
    plt.imshow(image)
    plt.show()


In [8]:
from keras.models import load_model
if model_try=='inceptionv3':
    pretrained_model = InceptionV3(input_shape = (img_width, img_height,3),include_top = False, weights = 'imagenet')
    for layer in pretrained_model.layers:
        layer.trainable = False

else:
    pretrained_model =VGG16(weights='imagenet', include_top=False, input_shape = (img_width, img_height, 3),pooling='avg')

 # Print the model summary
    pretrained_model.summary()
    pretrained_model.trainable = False
# 
model_v3 = Sequential()
model_v3.add(pretrained_model)
#pretrained_model)
# model_v3.add(Flatten())
model_v3.add(Dense(256,activation = 'relu'))
model_v3.add(Dropout(0.3))
model_v3.add(Dense(y_train.shape[1],activation = 'softmax'))
# model_v3 = load_model("weights/inceptionv3_150weights.h5")
model_v3.compile(optimizer = Adam(lr = 0.0001),loss = 'categorical_crossentropy',metrics = ['accuracy'])
model_v3.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [ ]:
#bottleneck_features_train = model_v3.predict_generator(train_generator, 512, verbose = 1)
# import numpy as np
# train_labels = np.array([0] * 32 + [1] * 32 + [2] * 32 + [3] * 32 + [4] * 32 + [5] * 32 + [6] * 32 + [7] * 32)
# bottleneck_features_validation = model_v3.predict_generator(val_generator, 64)
# validation_labels = np.array([0] * 8 + [1] * 8 + [2] * 8 + [3] * 8 + [4] * 8 + [5] * 8 + [6] * 8 + [7] * 8)




In [ ]:
callbacks_list = [
#         ModelCheckpoint(top_weights_path, monitor='val_accuracy', verbose=1, save_best_only=True),
#         EarlyStopping(monitor='val_acc', patience=5, verbose=0),
        ReduceLROnPlateau(monitor='val_accuracy',patience=3,verbose=1,factor=0.7)   ]
history = model_v3.fit_generator(train_generator, steps_per_epoch = 32, epochs=300,
                          validation_data=val_generator, validation_steps = 15, verbose=1,callbacks=callbacks_list)

Epoch 1/300
32/32 [==============================] - 117s 4s/step - loss: 2.1928 - accuracy: 0.1392 - val_loss: 2.0102 - val_accuracy: 0.1962
Epoch 2/300
32/32 [==============================] - 116s 4s/step - loss: 2.0973 - accuracy: 0.1304 - val_loss: 2.0542 - val_accuracy: 0.2500
Epoch 3/300
32/32 [==============================] - 126s 4s/step - loss: 2.0212 - accuracy: 0.2098 - val_loss: 1.9912 - val_accuracy: 0.2847
Epoch 4/300
32/32 [==============================] - 130s 4s/step - loss: 1.9832 - accuracy: 0.2353 - val_loss: 1.8533 - val_accuracy: 0.3465
Epoch 5/300
32/32 [==============================] - 138s 4s/step - loss: 1.9370 - accuracy: 0.2765 - val_loss: 1.9619 - val_accuracy: 0.3475
Epoch 6/300
32/32 [==============================] - 144s 5s/step - loss: 1.9113 - accuracy: 0.2824 - val_loss: 1.8151 - val_accuracy: 0.3985
Epoch 7/300
32/32 [==============================] - 142s 4s/step - loss: 1.8394 - accuracy: 0.3657 - val_loss: 1.8703 - val_accuracy: 0.3886
Epoch 

32/32 [==============================] - 172s 5s/step - loss: 1.3413 - accuracy: 0.6324 - val_loss: 1.7393 - val_accuracy: 0.5099
Epoch 53/300
32/32 [==============================] - 191s 6s/step - loss: 1.3418 - accuracy: 0.6314 - val_loss: 1.5334 - val_accuracy: 0.4894
Epoch 54/300
32/32 [==============================] - 192s 6s/step - loss: 1.3318 - accuracy: 0.6422 - val_loss: 1.4899 - val_accuracy: 0.4901

Epoch 00054: ReduceLROnPlateau reducing learning rate to 1.3841284044247003e-06.
Epoch 55/300
32/32 [==============================] - 175s 5s/step - loss: 1.3494 - accuracy: 0.6118 - val_loss: 1.6027 - val_accuracy: 0.4901
Epoch 56/300
32/32 [==============================] - 175s 5s/step - loss: 1.3569 - accuracy: 0.6294 - val_loss: 1.6730 - val_accuracy: 0.5124
Epoch 57/300
32/32 [==============================] - 163s 5s/step - loss: 1.3505 - accuracy: 0.6451 - val_loss: 1.5482 - val_accuracy: 0.4704

Epoch 00057: ReduceLROnPlateau reducing learning rate to 9.6888986718113

In [ ]:
generate_sol(model_try,model_v3)
# generate_sol("inceptionv3_150", model_v3)

In [ ]:
from sklearn.metrics import multilable_confusion_matrix


In [ ]:
pretrained_model.trainable = True
model_v3.summary()



In [ ]:
model_v3.compile(
    optimizer=Adam(1e-7),  # Low learning rate
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)

epochs = 10
model_v3.fit_generator(train_generator, epochs=epochs,
                          validation_data=val_generator, verbose=1)

In [ ]:
generate_sol("vgg_finetune", model_v3)
# generate_sol("inceptionv3_150_finetune", model_v3)